In [2]:
%cd ~/Documents/projects/stitch/python/
from stitch.core.stitch_frame import StitchFrame

%cd ~/Documents/projects/nerve/python/
from nerve.spec.aesthetics import *

import yaml
from plotly import graph_objs

/Users/alex/Documents/projects/stitch/python
/Users/alex/Documents/projects/nerve/python


In [3]:
config = '/Users/alex/Desktop/aesthetic-model.yml'
with open(config, 'r') as f:
    config = yaml.load(f)

app = config['appearance']
x = Appearance(app)
x.ground.palette[0][0]

'#242424'

In [168]:
import pandas as pd
from pandas import DataFrame, Series
import numpy as np

In [264]:
def get_graph_objs(print_=True):
    x = dir(graph_objs)
    x = filter(lambda x: re.search('^[A-Z]', x), x)
    names = sorted(list(x))
    x = [getattr(graph_objs, x) for x in names]
    x = Series(x)
    x = x.apply(lambda x: x.__doc__)
    x = x.apply(lambda x: re.split('\n\n', x)).tolist()
    x = DataFrame(x)
    mask = x[1].apply(pd.isnull)
    a = x[mask][0].apply(lambda x: ' '.join(x.split('\n')[1:-1]))
    mask = x[mask]
    x.loc[mask.index, 1] = a

    x[0] = names
    x = x[[0,1]]
    x[1] = x[1].apply(lambda x: re.sub('\n', '', x))
    x[1] = x[1].apply(lambda x: re.sub(',$', '', x))
    x[1] = x[1].apply(lambda x: re.sub(']$', '', x))
    x[1] = x[1].apply(lambda x: x + ']')
    mask = x[1].apply(lambda x: re.search('dict', x)).astype(bool)
    x = x[~mask]
    x[1] = x[1].apply(eval)
    x.sort_values(0, inplace=True)
    x.columns=['name', 'attr']
    x.fillna('[]', inplace=True)
    x.attr = x.attr.apply(str).apply(eval).apply(sorted)
    
    lut = set(list(chain(*x.attr.tolist())))
    lut.add('0')
    lut = sorted(lut)
    
    ilut = dict(enumerate(lut))
    lut = {v:k for k,v in ilut.items()}
    def func(words):
        output = np.zeros(347)
        for word in words:
            output[lut[word]] = lut[word]
        return output
    
    x['vector'] = x.attr.apply(func)
            
    if print_:
        for i, row in x.iterrows():
            print(row[0] + ':')
            for item in sorted(row[1]):
                print('\t', item)
    return x, lut, ilut
            
data, lut, ilut = get_graph_objs(False)

v = data.vector.apply(lambda x: [ilut[x] for x in x])
v = StitchFrame(v).flatten(dtype=list, prefix=False).to_dataframe()[:46].T
v.columns = data.name
v.index = v.index.to_series().apply(lambda x: ilut[x])
v = v.ix[1:]
v = v.applymap(lambda x: np.nan if x == '0' else x)
v['count'] = v.count(axis=1)
v.to_csv('/Users/alex/Desktop/plotly-graph-objs.csv')